In [15]:
import os 
from pathlib import Path
import cv2
import sys
from itertools import compress
import matplotlib.pyplot as plt

from InferenceDataset import *
from SiameseNetworkAPI import *

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.use_deterministic_algorithms(True)

In [16]:
%matplotlib inline

main_folder = '/Users/racheltay/Documents/school/cp5105-capstone-proj/data/'
obj_folder = main_folder+'ziwen/mouse/'
# video_path = '/Users/racheltay/Documents/school/cp5105-capstone-proj/data/inference_test/video/room1.mp4'
video_path = main_folder+'ziwen/test2.jpg'

threshold = 0.2

all_img_of_obj = load_images_from_folder(obj_folder)

cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
sample_rate = 1

for fno in range(0, total_frames, sample_rate):
	cap.set(cv2.CAP_PROP_POS_FRAMES, fno)
	_, image = cap.read()
	model = SiameseNetworkAPI(all_img_of_obj, image)
	xy_coords, conf_scores = model.inference()
	print('conf_scores', conf_scores)

	accepted = list(compress(xy_coords, [x > threshold for x in conf_scores]))
	if len(accepted) > 0:
		for coords in accepted:
			x1, y1, x2, y2 = coords
			x1, y1, x2, y2 = round(x1.item()), round(y1.item()), round(x2.item()), round(y2.item())
			plt.imshow(cv2.cvtColor(image[y1:y2, x1:x2], cv2.COLOR_BGR2RGB))
			plt.show()

Using cache found in /Users/racheltay/.cache/torch/hub/ultralytics_yolov5_master


In [ ]:
%matplotlib inline

# model = SiameseNetworkAPI(all_img_of_obj, image)
results = model.obj_detection_model(image, size=228).crop(save=False)

# model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True,  _verbose=False)
# # model.iou = 0.01

# results = model(image, size=288).crop(save=False)

print('objects found by yolov5')

for i in results: 
	x1, y1, x2, y2 = [round(x.item()) for x in i['box']]
	plt.imshow(cv2.cvtColor(image[y1:y2, x1:x2], cv2.COLOR_BGR2RGB))
	plt.show()

In [ ]:
# to print the frames of the video 
all_img_of_obj = load_images_from_folder(obj_folder)

cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
sample_rate = 20

for fno in range(0, total_frames, sample_rate):
	cap.set(cv2.CAP_PROP_POS_FRAMES, fno)
	_, image = cap.read()
	plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
	plt.show()